# 서울시 동별 아파트 연간 평균 매매가 구하기

### 로컬 db

In [1]:
import mysql
import mysql.connector

remote = mysql.connector.connect(
    host = 'localhost',
    port = '3306',
    user = 'root',
    password = 'alsgh9713',
    database = 'amrbase'
)

cur = remote.cursor()

### 프로젝트용 AWS db

In [ ]:
import mysql
import mysql.connector

remote = mysql.connector.connect(
    host = 'database-1.cp2kuggswzva.ap-northeast-2.rds.amazonaws.com',
    port = '3306',
    user = 'master',
    password = '0000',
    database = 'edaproject'
)

cur = remote.cursor()

### 크롤링 코드

In [4]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import UnexpectedAlertPresentException, NoSuchElementException, StaleElementReferenceException
import time
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import UnexpectedAlertPresentException
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from user_agent import generate_user_agent
import random

sql_insert_query = """
INSERT INTO SEOUL_DONG_PRICE_YEARLY (YEAR, GU, DONG, AVG_PRICE)
VALUES (%s, %s, %s, %s)
"""

options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={generate_user_agent()}')  # 무작위 User-Agent 설정


url = 'https://land.seoul.go.kr/land/rtms/rtmsApartment.do'  # 부동산 통계정보 사이트 url
small_driver = webdriver.Chrome(service=Service('/home/yoon/ws/eda/driver/chromedriver'),
                          options=options)

small_driver.get(url)

small_driver.maximize_window()  # 전체화면 크기로 지정

driver = small_driver

# 날짜 리스트 생성하기
# 2024년 08월 부터 2014년 01 월 까지 연도 기준 역순으로
start = 20240101
end = 20240831

start_year_list = []
end_year_list = []

for i in range(11):
    start_year_list.append(start)
    start -= 10000

#print('=' * 50)

for j in range(11):
    end_year_list.append(end)
    if end == 20240831:
        end -= 9600
    else:
        end -= 10000

start_date_box = driver.find_element(By.XPATH, '//*[@id="changeBgnde"]')  # 시작일 콤보박스
end_date_box = driver.find_element(By.XPATH, '//*[@id="changeEndde"]')  # 종료일 콤보박스

# 구 리스트 구하기
gu_list = []

gu_combo_box = driver.find_element(By.XPATH, '//*[@id="selectSigungu"]')

# gu_combo_box를 Select 클래스로 감싸기
select_gu = Select(gu_combo_box)

# 각 구 이름 리스트들 우선 구해놓고
gu_panel = gu_combo_box.find_elements(By.TAG_NAME, 'option')[1:]  # 구 이름 리스트

# 각 구에 해당하는 'value' 값 구해놓기
for idx, each_gu in enumerate(gu_panel):
    gu_code = each_gu.get_attribute('value')  # 구별 고유 코드
    gu_name = each_gu.text
    gu_list.append([gu_name, gu_code])

# 구 선택하기 ----------------------------------------------------------------------------------
for current_gu in gu_list:  # 원본
#for current_gu in gu_list[24:25]:  # 중랑구만 돌리기
    gu_code = current_gu[1]  # 각 구 코드
    gu_name = current_gu[0]
    select_gu.select_by_value(gu_code)  # 각 구를 선택해준다.
    time.sleep(0.2)

    # 구를 선택했다면 이제 동 선택하기 ----------------------------------------------------------------
    # 동 콤보박스에서
    dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.
    dong_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="selectBjdong"]')))
    select_dong = Select(dong_combo_box) 
    dong_panel = dong_combo_box.find_elements(By.TAG_NAME, 'option')[1:]
    # 해당 구의 동 선택하기
    dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.

    # 우선 현재 구의 동 html 리스트를 생성하고
    for idx, each_dong in enumerate(dong_panel):
        dong_code = each_dong.get_attribute('value')  # 동별 고유 코드
        dong_name = each_dong.text  # 동 이름
        dong_list.append([dong_name, dong_code])

    # 현재 구의 동 리스트에서 각 동을 하나씩 가져온다.
    for idx, dong in enumerate(dong_list):  # 원본
    #for idx, dong in enumerate(dong_list[41:]):  # 을지로 6가 부터 가져옴.
        code = dong[1]  # 동 코드
        name = dong[0]  # 동 이름
        select_dong.select_by_value(code)  # 각 동 클릭하기

        # 동까지 클릭했다면 날짜를 가져와야한다. ----------------------------------------------------------
        # 날짜 연 단위로 내려가는 함수
        for idx in range(len(start_year_list)):
            start_date_box = driver.find_element(By.XPATH, '//*[@id="changeBgnde"]')  # 시작일 콤보박스
            end_date_box = driver.find_element(By.XPATH, '//*[@id="changeEndde"]')  # 종료일 콤보박스

            start_date_box.send_keys(Keys.CONTROL, 'a', Keys.BACKSPACE)  # 시작 날짜 지우고
            start_date_box.send_keys(start_year_list[idx])  # 시작 날짜 보내기

            end_date_box.click()
            
            # 경고창이 뜬 경우 이를 처리하는 코드
            try:
                # 경고창이 나타나면 Alert 객체로 변환
                alert = Alert(driver)
                
                # 경고창 닫기 (확인 버튼 클릭)
                alert.accept()
                print("경고창이 닫혔습니다.")
            except:
                # 경고창이 없으면 예외 처리
                print("경고창이 없습니다.")

            end_date_box.send_keys(Keys.CONTROL, 'a', Keys.BACKSPACE)  # 끝나는 날짜 지우고
            end_date_box.send_keys(end_year_list[idx])  # 종료 날짜 보내기

            while True:
                # '검색' 클릭
                search_btn = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="search"]')))
                search_btn.click()
                # 경고창이 뜬 경우 이를 처리하는 코드
                try:
                    # 경고창이 나타나면 Alert 객체로 변환
                    alert = Alert(driver)
                    
                    # 경고창 닫기 (확인 버튼 클릭)
                    alert.accept()
                    print("경고창이 닫혔습니다.")
                except:
                    # 경고창이 없으면 넘어가고
                    print("경고창이 없습니다.")
                    break

            # 표 보일 때까지 스크롤
            element = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.ID, 'resultList2')))
            driver.execute_script("arguments[0].scrollIntoView();", element)

            # 표가 보이면
            WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.ID, 'resultList2')))

            # 무한 반복하면서 StaleElementReferenceException이 발생해도 계속 재시도
            while True:
                try:
                    time.sleep(0.2)
                    result_panel = driver.find_element(By.ID, 'resultList2')
                    tr_in_tbody_panel = driver.find_element(By.ID, 'resultList2').find_elements(By.TAG_NAME, 'tr')
                    td_in_first_tr = tr_in_tbody_panel[0].find_element(By.TAG_NAME, 'td').text
                    break  # 요소를 찾고 작업이 성공하면 while 루프 탈출

                except StaleElementReferenceException:
                    print("StaleElementReferenceException 발생, 재시도 중...")
                    time.sleep(0.1)  # 잠시 대기 후 다시 시도

            try:  # 이제 가격을 찾을 차례 ----------------------------------------------------------------------------
                if td_in_first_tr != '조회된 내용이 없습니다.':
                    # 표안의 글자가 보일 때 까지 기다리기
                    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, '//*[@id="resultList2"]/tr[1]/td[2]/a')))
                    # 현재 페이지 html 로 가져오기
                    page = driver.page_source
                    soup = BeautifulSoup(page, 'html.parser')

                    info_panel = soup.find('tbody', id='resultList2')  # 각 정보가 들어가 있는 표
                    info_list = info_panel.find_all('tr')  # 표 안의 모든 행을 가져옴

                    total_price = 0  # 가격 총합을 계산할 변수
                    price_count = 0  # 가격 항목 수를 계산할 변수

                    for idx, row in enumerate(info_list):
                        #WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CLASS_NAME, 'tb_bottom')))
                        info = row.find_all('td', class_='tb_bottom')  # 각 행의 'tb_bottom' 클래스 찾기
                        date = info[3].string  # 날짜
                        
                        year = int('20' + date[:2])  # 년도
                        #month = int(date[3:5])  # 월
                        price = int(info[4].string.replace(',', '')) * 10000  # 가격을 정수로 변환
                        total_price += price  # 가격 합산
                        price_count += 1  # 가격 항목 수 증가
                        #print(f'{gu_name} {name} 계약 날짜 {year}년 {month}월, 계약금액 {price}원')

                    # 가격 평균 계산 (거래가 있을 경우만 계산)
                    if price_count > 0:
                        avg_price = total_price // price_count
                        print(f'{gu_name} {name}의 {year}년 평균 계약금액: {avg_price}원')
                        cur.execute(sql_insert_query, (year, gu_name, name, avg_price))
                        remote.commit() 
                    else:
                        print(f'{gu_name} {name}에 대한 거래 정보가 없습니다.')
                    print('=' * 50)

                else:
                    print(f'{gu_name} {name}의 정보가 없습니다. 다음 동으로 넘어갑니다.')
                    print('=' * 50)
            
            except TimeoutException:  # 혹시나 화면 변환 과정에서 거래량이 없음에도 불구하고 timeoutexception 에러가 발생하면
                print(f'StaleElementReferenceException 발생...{gu_name} {name}의 정보가 없습니다. 다음 동으로 넘어갑니다.')
                print('=' * 50)


경고창이 없습니다.
경고창이 없습니다.
강남구 개포동의 2024년 평균 계약금액: 2225147377원
경고창이 닫혔습니다.
경고창이 없습니다.
강남구 개포동의 2023년 평균 계약금액: 1868879795원
경고창이 닫혔습니다.
경고창이 없습니다.
강남구 개포동의 2022년 평균 계약금액: 2130712121원
경고창이 닫혔습니다.
경고창이 없습니다.
강남구 개포동의 2021년 평균 계약금액: 2105346303원
경고창이 닫혔습니다.
경고창이 닫혔습니다.
경고창이 없습니다.
강남구 개포동의 2020년 평균 계약금액: 1780246013원
경고창이 닫혔습니다.
경고창이 없습니다.


KeyboardInterrupt: 

# 혹시나 크롤링 중 벤을 먹는다면....

- DB 확인 후 데이터 insert 가 끊긴 구나 동을 찾아 

- 해당 구나 동부터 다시 실행하기

- 셀레니움 창을 켠 상태에서 다음 코드를 실행하면 구, 동 리스트를 얻을 수 있음.

### 구 리스트 구하기

In [ ]:
gu_list = []

gu_combo_box = driver.find_element(By.XPATH, '//*[@id="selectSigungu"]')
gu_panel = gu_combo_box.find_elements(By.TAG_NAME, 'option')[1:]  # 구 이름 리스트

for idx, each_gu in enumerate(gu_panel):
    gu_code = each_gu.get_attribute('value')  # 구별 고유 코드
    gu_name = each_gu.text
    gu_list.append([gu_name,gu_code])

# 예를 들어 '도봉구' 부터 다시 해야 한다면 아와 같이 슬라이싱 후 
# 주 코드의 구 검색하는 부분에 추가
gu_list[9:10]

[['도봉구', '11320']]

### 동 리스트 구하기

In [12]:
dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.
dong_combo_box = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="selectBjdong"]')))
select_dong = Select(dong_combo_box) 
dong_panel = dong_combo_box.find_elements(By.TAG_NAME, 'option')[1:]
# 해당 구의 동 선택하기
dong_list = []  # 구가 초기화 될 때마다 dong_list 도 초기화 되야 함.

# 우선 현재 구의 동 html 리스트를 생성하고
for idx, each_dong in enumerate(dong_panel):
    dong_code = each_dong.get_attribute('value')  # 동별 고유 코드
    dong_name = each_dong.text  # 동 이름
    dong_list.append([dong_name, dong_code])

# 예를 현재 강남구, 삼성동부터 다시 해야 한다면 
# 강남구가 선택된 상태에서 이와 같이 슬라이싱 후 주 코드의 동 검색하는 부분에 추가
dong_list[4:5]

[['삼성동', '10500']]